In [2]:
import pandas as pd

In [1]:
%run AnalysisHelperFile.ipynb


Variables: put_sell_study_dict
           profit_put_sell_study_dict
           loss_put_sell_study_dict



In [3]:
for study in put_sell_study_dict:
    print(study, 'profit:', put_sell_study_dict[study]['PROFIT'].sum())

naked_50 profit: 91.43000000000002
naked_dte profit: 50.065
verticle_50 profit: 28.35
verticle_dte profit: 15.15


In [4]:
for study in profit_put_sell_study_dict:
    average_profit = (profit_put_sell_study_dict[study]['PROFIT'].sum() / len(profit_put_sell_study_dict[study]))
    print(study, 'profit:', average_profit)

naked_50 profit: 2.4321969696969696
naked_dte profit: 4.09875
verticle_50 profit: 1.0998360655737704
verticle_dte profit: 1.7695652173913041


In [5]:
for study in loss_put_sell_study_dict:
    average_loss = (loss_put_sell_study_dict[study]['PROFIT'].sum() / len(loss_put_sell_study_dict[study]))
    print(study, 'profit:', average_loss)

naked_50 profit: -13.818999999999999
naked_dte profit: -12.07625
verticle_50 profit: -5.534285714285715
verticle_dte profit: -5.109999999999999


# Calculating Vertical profit
profit is calculated by `Amount rec - Amount buyback`

What happens if we are comparing two different dataframe and one dataframe recieves more money?

When trading Vertical, amount received is based on the width(risk) of our trade
- `width = (sold strike - bought strike)`
- if the width is wider, more risk is added meaning we will receieve more money.

For example, if df1 contains mainly 5 points spread while df2 mainly has 10 points spread, df2 will have higher average money recieved and our normal profit equation will not fit into this situation
    
# How can we add width(risk) into consideration when calculating profit?

- ROI(Return of Capital) is calculated by (net profit / amount risked)
    - This equation can't be used on naked option since risk is unlimited
    - This is useful for Vertical trade since risk is determined
    
# ROI vs Profit
- ROI measures the % return on capital you used. That means the number can exceed 100%
- Profit measures the performance of your trade. It can not exceed 100%

In [6]:
def add_roi(df):
    output = df.copy()
    roi_list = []
    for row in range(len(df)):
        profit = df['PROFIT'][row]
        risk = df['AMOUNT_RISKED'][row]
        roi = (profit / risk).round(2)
        roi_list.append(roi)
    
    output = output.assign(ROI = roi_list)
    output = output.reset_index(drop=True)
    return output

Creates a dataframe with roi column and use that column to calculate the values we are looking for

In [7]:
roi_vert_50_df = add_roi(put_sell_study_dict['verticle_50'])
average_roi_vert_50 = (roi_vert_50_df['ROI'].sum() / len(roi_vert_50_df['ROI'])).round(2)

roi_vert_dte_df = add_roi(put_sell_study_dict['verticle_dte'])
average_roi_vert_dte = (roi_vert_dte_df['ROI'].sum() / len(roi_vert_dte_df['ROI'])).round(2)

performance_diff = (((average_roi_vert_dte - average_roi_vert_50) / average_roi_vert_50) * 100).round(2)

print('Average ROI for vert_50:', average_roi_vert_50)
print('Average ROI for vert_dte:', average_roi_vert_dte)
print('vert_dte performed', performance_diff,'% better than vert_50')

Average ROI for vert_50: 0.04
Average ROI for vert_dte: 0.08
vert_dte performed 100.0 % better than vert_50


In [8]:
profit_vert_50_df = put_sell_study_dict['verticle_50']['PROFIT']
average_profit_vert_50 = (profit_vert_50_df.sum() / len(profit_vert_50_df)).round(2)

profit_vert_dte_df = put_sell_study_dict['verticle_dte']['PROFIT']
average_profit_vert_dte = (profit_vert_dte_df.sum() / len(profit_vert_dte_df)).round(2)

performance_diff = (((average_profit_vert_dte - average_profit_vert_50) / average_profit_vert_50) * 100).round(2)

print('Average profit for vert_50:', average_profit_vert_50)
print('Average profit for vert_dte:', average_profit_vert_dte)
print('vert_dte performed', performance_diff,'% better than vert_50')

Average profit for vert_50: 0.42
Average profit for vert_dte: 0.54
vert_dte performed 28.57 % better than vert_50


It is expected that vert_dte would perform better than vert_50 because you are receiving more money upfront.

We have found that
- performance of Average ROI of vert_dte compared to vert_50 performed `100%` better than the performance of Average profit of vert_dte to vert_50.

# Takeaway
- Profit is used to calculate the performance of the trade not including how much risk you have taken
- ROI takes risk(width) into consideration and measures the % return of capital(same as risk) you have used.
### If the sole purpose of the trade is to get the highest ROI, naked trade can be a great trade choice.